In [ ]:
from agents import Agent, Runner, trace
from IPython.display import Markdown, display
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv(override=True)


In [ ]:
entrepreneur_instructions = "You generate novel, exciting business ideas for Autonomous AI Agents"
entrepreneur_input = """
Propose one compelling business idea for applying Autonomous AI Agents to any industry you choose.
The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
"""

evaluator_instructions = "You evaluate novel, exciting business ideas for Autonomous AI Agents"
evaluator_input = "Evaluate the following business idea; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents:\n\n"

In [ ]:
class Idea(BaseModel):
    title: str = Field(description="The title of the idea")
    description: str = Field(description="A detailed description of the idea, in Markdown")

    def print_summary(self):
        print(f"Title: {self.title}")
        display(Markdown(self.description))

In [ ]:
class Evaluation(BaseModel):
    idea: Idea = Field(description="The idea that you are evaluating")

    uniqueness_feedback: str = Field(description="Your view on how unique the idea is")
    uniqueness_score: int = Field(description="How unique the idea is, on a scale of 1 to 10, where 1 is mundane and 10 is groundbreaking")

    commercial_feedback: str = Field(description="Your view on the commercial potential of the idea")
    commercial_score: int = Field(description="How high the commercial potential of the idea is from 1 to 10, where 1 is barely profitable and 10 is a billion dollar idea")

    autonomy_feedback: str = Field(description="Your commentary on to what extent the idea truly benefits from the autonomous nature of AI Agents")
    autonomy_score: int = Field(description="How deeply the idea involves autonomous agents from 1 to 10, where 1 is that it plays no meaningful role, and 10 is that it is pivotal")

    def is_brilliant(self):
        return self.uniqueness_score > 8 and self.commercial_score > 8 and self.autonomy_score > 8

    def print_summary(self):
        print(f"Idea: {self.idea.title}")
        print(f"Uniqueness: {self.uniqueness_score}/10: ({self.uniqueness_feedback})")
        print(f"Commercial: {self.commercial_score}/10: ({self.commercial_feedback})")
        print(f"Autonomy: {self.autonomy_score}/10: ({self.autonomy_feedback})")

## Approach 1 (Highly Recommended!) - Call Runner.run() separately

In [ ]:
entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea)
result = await Runner.run(entrepreneur, entrepreneur_input)
idea = result.final_output_as(Idea)
idea.print_summary()

In [ ]:
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)
result = await Runner.run(evaluator, f"{evaluator_input}{idea}")
evaluation = result.final_output_as(Evaluation)
evaluation.print_summary()

In [ ]:
entrepreneur_latest = f"You were originally given the following task:\n\n{entrepreneur_input}\n\n"
remaining_attempts = 4

In [ ]:
with trace("Entrepreneur Approach 1"):
    while not evaluation.is_brilliant() and remaining_attempts:
        remaining_attempts -= 1

        # Build a composite prompt that includes:
        # - The original task
        # - The last idea and feedback
        # - Instructions to improve the idea
        entrepreneur_latest += (
            f"You responded with this idea:\n\n{idea}\n\n"
            f"You received this feedback:\n\n{evaluation}\n\n"
            "Now respond with an improved idea that addresses the feedback. "
            "Do not directly reference the feedback.\n\n"
        )

        # 🔹 Step 1: Entrepreneur proposes an improved idea
        result = await Runner.run(entrepreneur, entrepreneur_latest)
        idea = result.final_output_as(Idea)
        idea.print_summary()

        # 🔹 Step 2: Evaluator evaluates the new idea
        result = await Runner.run(evaluator, f"{evaluator_input}{idea}")
        evaluation = result.final_output_as(Evaluation)
        evaluation.print_summary()

## 🧠 Key Things to Notice

In [ ]:
"""
Element	                    Purpose
Agent(...)	                Defines a role and behavior (Entrepreneur or Evaluator). Each is an LLM with a specific set of instructions.
Runner.run(agent, input)	  Executes one turn for the agent — sending the input and collecting the structured result.
Idea / Evaluation	          Typed outputs using Pydantic models, so you can programmatically access structured fields (idea.title, evaluation.commercial_score, etc.).
evaluation.is_brilliant()	  Simple logic gate to determine when to stop iterating.
with trace("..."):	        Creates a trace span that groups all of these interactions together for analysis, logging, or visualization.
"""